# Exporting Landsat 7 to GeoTiff format  

> **Description**  
> The code in this notebook subsets a data cube, selects a specific set of variables, and then outputs that data into a GeoTIFF file. The goal is to be able to do external analyses of this data using other data analysis tools or GIS tools. The files would be reasonable in size, since we would restrict the region and parameters in the output.

----  

# Boiler Plate, Loading Data

> ### Import the Datacube

In [1]:
import datacube
dc = datacube.Datacube(app = 'my_app', config = '/home/localuser/.datacube.conf')

/home/localuser/Datacube/datacube_env/lib/python3.6/site-packages/psycopg2-2.7.5-py3.6-linux-x86_64.egg/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


>### Browse the available Data Cubes on the storage platform    
> You might want to learn more about what data is stored and how it is stored.


In [2]:
list_of_products = dc.list_products()
netCDF_products = list_of_products[list_of_products['format'] == 'NetCDF']
netCDF_products

,name,description,lat,product_type,creation_time,label,platform,time,instrument,lon,format,crs,resolution,tile_size,spatial_dimensions
id,,,,,,,,,,,,,,,
13,ls7_ledaps_ghana,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,None,None,LANDSAT_7,None,ETM,None,NetCDF,EPSG:4326,"(-0.000269494585236, 0.000269494585236)","(0.943231048326, 0.943231048326)","(latitude, longitude)"
17,ls7_ledaps_kenya,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,None,None,LANDSAT_7,None,ETM,None,NetCDF,EPSG:4326,"(-0.000269493, 0.000269493)","(0.99981903, 0.99981903)","(latitude, longitude)"
18,ls7_ledaps_senegal,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,None,None,LANDSAT_7,None,ETM,None,NetCDF,EPSG:4326,"(-0.000271152, 0.00027769)","(0.813456, 0.83307)","(latitude, longitude)"
16,ls7_ledaps_sierra_leone,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,None,None,LANDSAT_7,None,ETM,None,NetCDF,EPSG:4326,"(-0.000269494585236, 0.000269494585236)","(0.943231048326, 0.943231048326)","(latitude, longitude)"
19,ls7_ledaps_tanzania,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,None,None,LANDSAT_7,None,ETM,None,NetCDF,EPSG:4326,"(-0.000271277688070265, 0.000271139577954979)","(0.999929558226998, 0.999962763497961)","(latitude, longitude)"
31,ls7_ledaps_vietnam,Landsat 7 USGS Collection 1 Higher Level SR sc...,None,LEDAPS,None,None,LANDSAT_7,None,ETM,None,NetCDF,EPSG:4326,"(-0.000269494585236, 0.000269494585236)","(0.943231048326, 0.943231048326)","(latitude, longitude)"
9,ls8_lasrc_ghana,Landsat 8 USGS Collection 1 Higher Level SR sc...,None,LaSRC,None,None,LANDSAT_8,None,OLI_TIRS,None,NetCDF,EPSG:4326,"(-0.000269494585236, 0.000269494585236)","(0.943231048326, 0.943231048326)","(latitude, longitude)"
10,ls8_lasrc_kenya,Landsat 8 USGS Collection 1 Higher Level SR sc...,None,LaSRC,None,None,LANDSAT_8,None,OLI_TIRS,None,NetCDF,EPSG:4326,"(-0.000271309115317046, 0.00026957992707863)","(0.999502780827996, 0.999602369607559)","(latitude, longitude)"
11,ls8_lasrc_senegal,Landsat 8 USGS Collection 1 Higher Level SR sc...,None,LaSRC,None,None,LANDSAT_8,None,OLI_TIRS,None,NetCDF,EPSG:4326,"(-0.000271152, 0.00027769)","(0.813456, 0.83307)","(latitude, longitude)"


>### Pick a product  
>Use the platform names from the previous block to select a small Data Cube. The data_access_api utility will give you lat, lon, and time bounds of your Data Cube.   

In [3]:
import utils.data_cube_utilities.data_access_api as dc_api  
api = dc_api.DataAccessApi(config = '/home/localuser/.datacube.conf')

platform = "LANDSAT_7"
product = "ls7_ledaps_vietnam"

# Get product extents
prod_extents = api.get_query_metadata(platform=platform, product=product, measurements=[])#.get_full_dataset_extent(platform = platform, product = product)

latitude_extents = prod_extents['lat_extents']
print("Lat bounds:", latitude_extents)
longitude_extents = prod_extents['lon_extents']
print("Lon bounds:", longitude_extents)
time_extents = list(map(lambda time: time.strftime('%Y-%m-%d'), prod_extents['time_extents']))
print("Time bounds:", time_extents)

Lat bounds: (9.1762906272858, 13.964939912344285)
Lon bounds: (102.4041694654867, 108.9310588253174)
Time bounds: ['1999-09-08', '2016-12-29']


/home/localuser/Datacube/data_cube_notebooks/utils/data_cube_utilities/data_access_api.py:184: FutureWarning: casting an xarray.Dataset to a boolean will change in xarray v0.11 to only include data variables, not coordinates. Cast the Dataset.variables property instead to preserve existing behavior in a forwards compatible manner.
  if not dataset:


# Visualize Data Cube Region

> #### Picking a smaller analysis region

In [4]:
## The code below renders a map that can be used to orient yourself with the region.
from utils.data_cube_utilities.dc_display_map import display_map
display_map(latitude = latitude_extents, longitude = longitude_extents)

In [5]:
######### Vietnam - Buan Tua Srah Lake ################## 
longitude_extents = (108.02, 108.15)
latitude_extents  = (12.18 , 12.30)

time_extents = ('2015-01-01', '2016-01-01')
print ( time_extents )

('2015-01-01', '2016-01-01')


In [6]:
from utils.data_cube_utilities.dc_display_map import display_map

display_map(latitude = latitude_extents, longitude = longitude_extents)

In [7]:
landsat_dataset = dc.load(latitude = latitude_extents,
                          longitude = longitude_extents,
                          platform = platform,
                          time = time_extents,
                          product = product,
                          measurements = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'pixel_qa']) 

In [8]:
landsat_dataset

<xarray.Dataset>
Dimensions:    (latitude: 446, longitude: 483, time: 19)
Coordinates:
  * time       (time) datetime64[ns] 2015-01-09T03:06:13 ... 2015-12-27T03:08:59
  * latitude   (latitude) float64 12.3 12.3 12.3 12.3 ... 12.18 12.18 12.18
  * longitude  (longitude) float64 108.0 108.0 108.0 108.0 ... 108.1 108.1 108.1
Data variables:
    red        (time, latitude, longitude) int16 912 1008 1125 ... 614 464 427
    green      (time, latitude, longitude) int16 674 779 842 863 ... 508 487 467
    blue       (time, latitude, longitude) int16 493 473 574 554 ... 390 293 312
    nir        (time, latitude, longitude) int16 2500 2544 2587 ... 2772 2731
    swir1      (time, latitude, longitude) int16 2740 3002 3054 ... 2095 1993
    swir2      (time, latitude, longitude) int16 1678 1982 2037 ... 1099 991
    pixel_qa   (time, latitude, longitude) int32 66 66 66 66 66 ... 66 66 66 66
Attributes:
    crs:      EPSG:4326

# Derive Several Products

>### Unpack pixel_qa

In [9]:
import xarray as xr  
import numpy as np

def ls7_unpack_qa( data_array , cover_type):  
    
    land_cover_endcoding = dict( fill     =  [1], 
                                 clear    =  [66,  130], 
                                 water    =  [68,  132],
                                 shadow   =  [72,  136],
                                 snow     =  [80,  112, 144, 176],
                                 cloud    =  [96,  112, 160, 176, 224],
                                 low_conf =  [66,  68,  72,  80,  96,  112],
                                 med_conf =  [130, 132, 136, 144, 160, 176],
                                 high_conf=  [224]
                               ) 
    boolean_mask = np.isin(data_array.values, land_cover_endcoding[cover_type]) 
    return xr.DataArray(boolean_mask.astype(int),
                        coords = data_array.coords,
                        dims = data_array.dims,
                        name = cover_type + "_mask",
                        attrs = data_array.attrs)  

In [10]:
clear_xarray  = ls7_unpack_qa(landsat_dataset.pixel_qa, "clear")  
water_xarray  = ls7_unpack_qa(landsat_dataset.pixel_qa, "water")

shadow_xarray = ls7_unpack_qa(landsat_dataset.pixel_qa, "shadow")  

In [11]:
clean_xarray = xr.ufuncs.logical_or(clear_xarray , water_xarray).astype(np.int8).rename("clean_mask")

clean_mask = np.logical_or(clear_xarray.values.astype(bool),
                           water_xarray.values.astype(bool)) 

> ### Water

In [12]:
from utils.data_cube_utilities.dc_water_classifier import wofs_classify

water_classification = wofs_classify(landsat_dataset,
                                     clean_mask = clean_mask, 
                                     mosaic = False) 

/home/localuser/Datacube/data_cube_notebooks/utils/data_cube_utilities/dc_water_classifier.py:117: RuntimeWarning: divide by zero encountered in true_divide
  return (a - b) / (a + b)


In [13]:
wofs_xarray = water_classification.wofs

> ###  Normalized Indices  

In [14]:
def NDVI(dataset):
    return ((dataset.nir - dataset.red)/(dataset.nir + dataset.red)).rename("NDVI")

In [15]:
def NDWI(dataset):
    return ((dataset.green - dataset.nir)/(dataset.green + dataset.nir)).rename("NDWI")

In [16]:
def NDBI(dataset):
        return ((dataset.swir2 - dataset.nir)/(dataset.swir2 + dataset.nir)).rename("NDBI")

In [17]:
ndbi_xarray = NDBI(landsat_dataset)  # Urbanization - Reds
ndvi_xarray = NDVI(landsat_dataset)  # Dense Vegetation - Greens
ndwi_xarray = NDWI(landsat_dataset)  # High Concentrations of Water - Blues  

>### TSM  

In [18]:
from utils.data_cube_utilities.dc_water_quality import tsm

tsm_xarray = tsm(landsat_dataset, clean_mask = wofs_xarray.values.astype(bool) ).tsm

> ### EVI  

In [19]:
def EVI(dataset, c1 = None, c2 = None, L = None):
        return ((dataset.nir - dataset.red)/((dataset.nir  + (c1 * dataset.red) - (c2 *dataset.blue) + L))).rename("EVI")

In [20]:
evi_xarray = EVI(landsat_dataset, c1 = 6, c2 = 7.5, L = 1 )

# Combine Everything  

In [21]:
combined_dataset = xr.merge([landsat_dataset,
          clean_xarray,
          clear_xarray,
          water_xarray,
          shadow_xarray,
          evi_xarray,
          ndbi_xarray,
          ndvi_xarray,
          ndwi_xarray,
          wofs_xarray,
          tsm_xarray])

# Copy original crs to merged dataset 
combined_dataset = combined_dataset.assign_attrs(landsat_dataset.attrs)

combined_dataset

<xarray.Dataset>
Dimensions:      (latitude: 446, longitude: 483, time: 19)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-09T03:06:13 ... 2015-12-27T03:08:59
  * latitude     (latitude) float64 12.3 12.3 12.3 12.3 ... 12.18 12.18 12.18
  * longitude    (longitude) float64 108.0 108.0 108.0 ... 108.1 108.1 108.1
Data variables:
    red          (time, latitude, longitude) float32 912.0 1008.0 ... 427.0
    green        (time, latitude, longitude) float32 674.0 779.0 ... 487.0 467.0
    blue         (time, latitude, longitude) float32 493.0 473.0 ... 293.0 312.0
    nir          (time, latitude, longitude) float32 2500.0 2544.0 ... 2731.0
    swir1        (time, latitude, longitude) float32 2740.0 3002.0 ... 1993.0
    swir2        (time, latitude, longitude) float32 1678.0 1982.0 ... 991.0
    pixel_qa     (time, latitude, longitude) int32 66 66 66 66 ... 66 66 66 66
    clean_mask   (time, latitude, longitude) int8 1 1 1 1 1 1 1 ... 1 1 1 1 1 1
    clear_mask   (time, lat

# Export Geotiff

----  
File formatting  

In [22]:
import time
def time_to_string(t):
    return time.strftime("%Y_%m_%d_%H_%M_%S", time.gmtime(t.astype(int)/1000000000))

----  
Start Export

In [23]:
# Ensure the output directory exists before writing to it.
!mkdir -p output/geotiffs/landsat7

from utils.data_cube_utilities.import_export import export_xarray_to_geotiff
export_xarray_to_geotiff(combined_dataset, "output/geotiffs/landsat7/landsat7")

----  
Sanity check using `gdalinfo` to make sure that all of our bands exist    

In [24]:
!gdalinfo output/geotiffs/landsat7/landsat7_2015_01_09_03_06_13.tif

Driver: GTiff/GeoTIFF
Files: output/geotiffs/landsat7/landsat7_2015_01_09_03_06_13.tif
Size is 483, 446
Coordinate System is `'
Origin = (108.020032379927088,12.299867617463660)
Pixel Size = (0.000268936625432,-0.000268890337287)
Image Structure Metadata:
  INTERLEAVE=PIXEL
Corner Coordinates:
Upper Left  ( 108.0200324,  12.2998676) 
Lower Left  ( 108.0200324,  12.1799425) 
Upper Right ( 108.1499288,  12.2998676) 
Lower Right ( 108.1499288,  12.1799425) 
Center      ( 108.0849806,  12.2399051) 
Band 1 Block=483x1 Type=Float32, ColorInterp=Gray
  NoData Value=-9999
Band 2 Block=483x1 Type=Float32, ColorInterp=Undefined
  NoData Value=-9999
Band 3 Block=483x1 Type=Float32, ColorInterp=Undefined
  NoData Value=-9999
Band 4 Block=483x1 Type=Float32, ColorInterp=Undefined
  NoData Value=-9999
Band 5 Block=483x1 Type=Float32, ColorInterp=Undefined
  NoData Value=-9999
Band 6 Block=483x1 Type=Float32, ColorInterp=Undefined
  NoData Value=-9999
Band 7 Block=483x1 Type=Float32, ColorInterp=Unde

----  
Check to see what files exist in `geotiffs`

In [25]:
!ls -lah output/geotiffs/landsat7/*.tif

-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_01_09_03_06_13.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_01_25_03_06_16.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_02_10_03_06_21.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_02_26_03_06_30.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_03_14_03_06_36.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_03_30_03_06_43.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_04_15_03_06_52.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_05_01_03_06_58.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_05_17_03_07_06.tif
-rw-rw-r-- 1 localuser local

----  
Zip all geotiffs  

In [26]:
!tar -cvzf output/geotiffs/landsat7/landsat_7.tar.gz output/geotiffs/landsat7/*.tif

output/geotiffs/landsat7/landsat7_2015_01_09_03_06_13.tif
output/geotiffs/landsat7/landsat7_2015_01_25_03_06_16.tif
output/geotiffs/landsat7/landsat7_2015_02_10_03_06_21.tif
output/geotiffs/landsat7/landsat7_2015_02_26_03_06_30.tif
output/geotiffs/landsat7/landsat7_2015_03_14_03_06_36.tif
output/geotiffs/landsat7/landsat7_2015_03_30_03_06_43.tif
output/geotiffs/landsat7/landsat7_2015_04_15_03_06_52.tif
output/geotiffs/landsat7/landsat7_2015_05_01_03_06_58.tif
output/geotiffs/landsat7/landsat7_2015_05_17_03_07_06.tif
output/geotiffs/landsat7/landsat7_2015_06_02_03_07_11.tif
output/geotiffs/landsat7/landsat7_2015_08_05_03_07_28.tif
output/geotiffs/landsat7/landsat7_2015_08_21_03_07_30.tif
output/geotiffs/landsat7/landsat7_2015_09_06_03_07_32.tif
output/geotiffs/landsat7/landsat7_2015_10_08_03_07_43.tif
output/geotiffs/landsat7/landsat7_2015_10_24_03_08_01.tif
output/geotiffs/landsat7/landsat7_2015_11_09_03_08_15.tif
output/geotiffs/landsat7/landsat7_2015_11_25_03_08_31.tif
output/geotiff

----  
List files again to see the size of the gif created

In [29]:
!ls -lah output/geotiffs/landsat7/*.tif

-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_01_09_03_06_13.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_01_25_03_06_16.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_02_10_03_06_21.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_02_26_03_06_30.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_03_14_03_06_36.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_03_30_03_06_43.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_04_15_03_06_52.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_05_01_03_06_58.tif
-rw-rw-r-- 1 localuser localuser 14M Nov  2 20:02 output/geotiffs/landsat7/landsat7_2015_05_17_03_07_06.tif
-rw-rw-r-- 1 localuser local

In [28]:
!gdalinfo output/geotiffs/landsat7/landsat7_2015_08_05_03_07_28.tif

Driver: GTiff/GeoTIFF
Files: output/geotiffs/landsat7/landsat7_2015_08_05_03_07_28.tif
Size is 483, 446
Coordinate System is `'
Origin = (108.020032379927088,12.299867617463660)
Pixel Size = (0.000268936625432,-0.000268890337287)
Image Structure Metadata:
  INTERLEAVE=PIXEL
Corner Coordinates:
Upper Left  ( 108.0200324,  12.2998676) 
Lower Left  ( 108.0200324,  12.1799425) 
Upper Right ( 108.1499288,  12.2998676) 
Lower Right ( 108.1499288,  12.1799425) 
Center      ( 108.0849806,  12.2399051) 
Band 1 Block=483x1 Type=Float32, ColorInterp=Gray
  NoData Value=-9999
Band 2 Block=483x1 Type=Float32, ColorInterp=Undefined
  NoData Value=-9999
Band 3 Block=483x1 Type=Float32, ColorInterp=Undefined
  NoData Value=-9999
Band 4 Block=483x1 Type=Float32, ColorInterp=Undefined
  NoData Value=-9999
Band 5 Block=483x1 Type=Float32, ColorInterp=Undefined
  NoData Value=-9999
Band 6 Block=483x1 Type=Float32, ColorInterp=Undefined
  NoData Value=-9999
Band 7 Block=483x1 Type=Float32, ColorInterp=Unde